## Notebook for augmenting images used for first stage YOLO model of 2-stage model pipline

In [8]:
# Check if all image have label or vice versa
import os

image_dir = "TwoStageYOLODataset/original/images"
label_dir = "TwoStageYOLODataset/original/labels"

# 获取不含扩展名的文件名集合
image_files = {os.path.splitext(f)[0] for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.png'))}
label_files = {os.path.splitext(f)[0] for f in os.listdir(label_dir) if f.lower().endswith('.txt')}

# 差集
images_without_labels = image_files - label_files
labels_without_images = label_files - image_files

# 打印结果
print(f"🔍 图像总数: {len(image_files)}")
print(f"📝 标签总数: {len(label_files)}\n")

print(f"📸 有图像但没有标签的数量: {len(images_without_labels)}")
for name in sorted(images_without_labels):
    print(f" - {name}.jpg")

print(f"\n🗂️ 有标签但没有图像的数量: {len(labels_without_images)}")
for name in sorted(labels_without_images):
    print(f" - {name}.txt")


🔍 图像总数: 139
📝 标签总数: 139

📸 有图像但没有标签的数量: 0

🗂️ 有标签但没有图像的数量: 0


In [19]:
import os, itertools, copy
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm
import augment_utils_chain as aug

# 子集名称
subsets = ['train', 'val', 'test']

# 增强函数
AUG_FUNCS = [
    aug.add_gaussian_noise,
    aug.adjust_random_brightness,
    aug.add_black_rect,
    aug.horizontal_flip,
    aug.random_rotate,
    aug.random_scale_with_padding,
]

# 不建议同时用的组合
bad_pair = {aug.add_gaussian_noise, aug.random_scale_with_padding}


def process_one(name):
    try:
        img, boxes, cls, _, _ = aug.load_image_and_boxes(name)
        aug.save_augmented(img, boxes, cls, name)
        for combo in itertools.combinations(AUG_FUNCS, 4):
            if bad_pair.issubset(combo):
                continue
            img_aug, boxes_aug, cls_aug = copy.deepcopy(img), copy.deepcopy(boxes), cls
            for fn in combo:
                img_aug, boxes_aug, cls_aug = fn(img_aug, boxes_aug, cls_aug)
            aug.save_augmented(img_aug, boxes_aug, cls_aug, name)
    except Exception as e:
        print(f"[ERROR] {name}: {e}")


for subset in subsets:
    print(f"\n🚀 Processing {subset} set...")

    # 设置路径
    aug.INPUT_DIR = f"TwoStageYOLODataset/squared"
    aug.OUTPUT_DIR = f"TwoStageYOLODataset/augmented"

    image_subdir = os.path.join(aug.INPUT_DIR, "images", subset)
    label_subdir = os.path.join(aug.INPUT_DIR, "labels", subset)

    # 重写 load/save 函数需要 subset 上下文 —— 我们把 subset 添加进文件名
    img_names = [
        os.path.join(subset, os.path.splitext(f)[0])
        for f in os.listdir(image_subdir)
        if f.lower().endswith(".jpg")
    ]

    # 多线程处理
    with ThreadPoolExecutor(max_workers=20) as ex:
        list(tqdm(ex.map(process_one, img_names), total=len(img_names)))



🚀 Processing train set...


  0%|          | 0/111 [00:00<?, ?it/s]


🚀 Processing val set...


  0%|          | 0/13 [00:00<?, ?it/s]


🚀 Processing test set...


  0%|          | 0/15 [00:00<?, ?it/s]

In [9]:
# Convert to square image of 1440*1440
from PIL import Image, ImageOps
import os

def pad_to_square_and_update_yolo(img_path, label_path, output_img_dir, output_label_dir):
    img = Image.open(img_path)
    w, h = img.size  # Should be 1080x1440

    # Padding: add 180 pixels left/right to make it square
    delta_w = 1440 - w  # 360
    pad_left = delta_w // 2
    pad_right = delta_w - pad_left
    padding = (pad_left, 0, pad_right, 0)  # (left, top, right, bottom)
    padded_img = ImageOps.expand(img, padding, fill=(0, 0, 0))

    # Save padded image
    os.makedirs(output_img_dir, exist_ok=True)
    img_filename = os.path.basename(img_path)
    padded_img.save(os.path.join(output_img_dir, img_filename))

    # Adjust YOLO labels
    os.makedirs(output_label_dir, exist_ok=True)
    new_label_path = os.path.join(output_label_dir, os.path.basename(label_path))

    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            lines = f.readlines()

        new_lines = []
        for line in lines:
            cls, x, y, w_box, h_box = map(float, line.strip().split())

            # Convert to absolute coordinates
            x_abs = x * w
            y_abs = y * h
            w_abs = w_box * w
            h_abs = h_box * h

            # Shift x due to padding
            x_abs += pad_left

            # New normalized values (on 1440x1440)
            x_new = x_abs / 1440
            y_new = y_abs / 1440
            w_new = w_abs / 1440
            h_new = h_abs / 1440

            new_line = f"{int(cls)} {x_new:.6f} {y_new:.6f} {w_new:.6f} {h_new:.6f}"
            new_lines.append(new_line)

        with open(new_label_path, 'w') as f:
            f.write('\n'.join(new_lines))

        print(f"Padded + updated: {img_filename}")



input_img_dir = 'TwoStageYOLODataset/original/images'           # ← 输入图片路径
input_label_dir = 'TwoStageYOLODataset/original/labels'         # ← 输入 YOLO 标签路径（.txt）
output_img_dir = 'TwoStageYOLODataset/squared/images'         # ← 输出图片路径
output_label_dir = 'TwoStageYOLODataset/squared/labels'       # ← 输出标签路径

# 创建输出目录
os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)


# Call function for all images
for filename in os.listdir(input_img_dir):
    if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue
    img_path = os.path.join(input_img_dir, filename)
    label_path = os.path.join(input_label_dir, os.path.splitext(filename)[0] + '.txt')
    
    pad_to_square_and_update_yolo(
        img_path,
        label_path,
        output_img_dir,
        output_label_dir
    )

Padded + updated: 0024e2fb-IMG_20250504_163554.jpg
Padded + updated: 044fe449-ca5db8ce3124515c2bb4cffbf83680d.jpg
Padded + updated: 052b5e49-38ee20756033d13b0a5de48a476f372.jpg
Padded + updated: 070cd6a7-IMG_20250515_114615.jpg
Padded + updated: 07aa7552-20250510_183822.jpg
Padded + updated: 07ce01d0-ecc48860-IMG_20250427_165039.jpg
Padded + updated: 0ed66e0a-IMG_20250506_181958.jpg
Padded + updated: 11dbab4c-602c2017-IMG_20250427_165342.jpg
Padded + updated: 13e313c1-IMG_20250504_161424.jpg
Padded + updated: 159e6cac-39a73c67c92fd9725c0d3a905edc124.jpg
Padded + updated: 15e5e754-20250502_173131.jpg
Padded + updated: 164a9ef9-f5e87f52272f6ea87994a5e548b5b1c.jpg
Padded + updated: 17ff7546-5e168810-IMG_20250427_151613.jpg
Padded + updated: 1b30ad36-acad2ddd8f04a9227d348fb8d41f296.jpg
Padded + updated: 1d2253c9-b91cd0831e5e566042fca6d21900b2a.jpg
Padded + updated: 1e977f20-cef8710a-IMG_20250427_152010.jpg
Padded + updated: 222df708-894876bf-IMG_20250427_165035.jpg
Padded + updated: 22e3fd